In [1]:
import numpy as np
import pandas as pd
import math


#Use to import the file into google Colab drive

In [2]:
bond_df = pd.read_excel('Dataset.xlsx', sheet_name='Bond')
mortgage_df = pd.read_excel('Dataset.xlsx', sheet_name = "Mortgage")
retailloan_df = pd.read_excel('Dataset.xlsx', sheet_name = "RetailLoan")

In [3]:
bond_df.head()
mortgage_df.head()

,ID,OriginalAmount,TermYears,Collateral_valuation,CurrentInstallment,Rate,PD,LGD
0,18101,2067863,35,678311,306,0.082,0.004728,0.497927
1,7835,309683,22,259913,166,0.117,0.176124,0.278921
2,26416,2998421,21,2218432,133,0.092,0.014447,0.570983
3,7304,483754,23,246967,152,0.032,0.168241,0.405557
4,14807,1808956,37,499590,93,0.114,0.006678,0.457563


In [4]:
#Market rates = 1.42 on October 4th.
# Parameters
principal = bond_df.Principal/100 #Principal
yearly_rate = bond_df.YearlyRate #Yearly Rate
time_maturity = bond_df.Time2Maturity

# Payment sequence
current_coupon = bond_df.CurrentCoupon
coupon_remain = bond_df.CouponsRemaining
time_maturity = bond_df.Time2Maturity
n = len(bond_df)

coupon_rate = np.zeros(n)
coupon = np.zeros(n)
bond_price = np.zeros(n)
number_coupon_year = np.zeros(n)
bonds_held = bond_df.BondsHeld

for i in range (n):
    number_coupon_year[i] = coupon_remain[i]/time_maturity[i]
    coupon_rate[i] = (yearly_rate[i]/100)/ number_coupon_year[i]
    coupon[i] = coupon_rate[i] * principal[i]
    #print(coupon[i])
    #payment_series = np.zeros(coupon_remain[i])
    payment_series = np.repeat(coupon[i], coupon_remain[i]+1)
    payment_series[0] -= coupon[i]
    payment_series[coupon_remain[i]] += principal[i]  # Last payment with principal
    if 1 < time_maturity[i] <= 3:
        r = 0.0148
    if 3 < time_maturity[i] <= 5:
        r = 0.0132
    if 5 < time_maturity[i] <= 10:
        r = 0.0128
    if time_maturity[i] > 10:
        r = 0.0144
    bond_price[i] = np.npv(r, payment_series)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


In [5]:
bond_price

array([0.83647776, 0.8547267 , 0.50582462, ..., 0.5611738 , 0.84865866,
       0.53285758])

In [6]:
#Part 1b Expected Loss
#Calculate the total provision of bond: expected loss = PD*LGD*EAD
#Senior claims on corporates, sovereigns and banks not secured by recognized collateral attract a 45% LGD.
#All subordinated claims on corporates, sovereigns and banks attract a 75% LGD.
bond_PD = bond_df.PD
bond_EAD = np.zeros(n)
bond_LGD = np.zeros(n)
bond_provision = np.zeros(n)

for i in range(n):
    if bond_PD[i] < 0.36:
        bond_LGD[i] =  0.45
    else:
        bond_LGD[i] = 0.75

for i in range(n):
    bond_EAD[i] = bond_price[i] * bonds_held[i]
    if math.isnan(bond_EAD[i]):
        bond_EAD[i] = 0
    bond_provision[i] = bond_PD[i] * bond_LGD[i] * bond_EAD[i]

    

In [7]:
print(np.sum(bond_provision))

4878586623.083627


In [8]:
#1b
#Calculate the total provision of mortgage: expected loss = PD*LGD*EAD
mortgage_PD = mortgage_df.PD
mortgage_LGD = mortgage_df.LGD

#Find mortgage_EAD
#Step 1: find m
mortgage_yearly_rate = mortgage_df.Rate
mortgage_times = mortgage_df.TermYears
mortgage_orgamt = mortgage_df.OriginalAmount
mortgage_rate = mortgage_yearly_rate/12
collateral = mortgage_df.Collateral_valuation
mortgage_len = len(mortgage_df)
m = np.zeros(mortgage_len)
mortgage_EAD = np.zeros(mortgage_len)
mortgage_provision = np.zeros(mortgage_len)
current_pricipal = np.zeros(mortgage_len)
#Step 2: find latest principal
for i in range(mortgage_len):
    m[i] = np.pmt(mortgage_rate[i], mortgage_times[i], mortgage_orgamt[i])
    current_pricipal[i] = np.fv(mortgage_rate[i], mortgage_times[i], m[i], -mortgage_orgamt[i])
    mortgage_EAD[i] = current_pricipal[i] - collateral [i]
    mortgage_provision[i] = mortgage_PD[i] * mortgage_LGD[i] * mortgage_EAD[i]

In [9]:
print(np.sum(mortgage_provision))

6450959903.849273


In [10]:
#Part 1b
#Calculate loan provision for retail loan
retailloan_PD = retailloan_df.PD
retailloan_LGD = 1 #Unsecured

#Find retailloan_EAD
#Step 1: find m
retailloan_yearly_rate = retailloan_df.YearlyInterestRate
retailloan_times = retailloan_df.Term
retailloan_orgamt = retailloan_df.OriginalAmount
retailloan_rate = mortgage_yearly_rate/12
retailloan_len = len(retailloan_df)
m2 = np.zeros(retailloan_len)
retailloan_EAD = np.zeros(retailloan_len)
retailloan_provision = np.zeros(retailloan_len)
current_pricipal2 = np.zeros(retailloan_len)
#Step 2: find latest principal
for i in range(retailloan_len):
    m2[i] = np.pmt(retailloan_rate[i], retailloan_times[i], retailloan_orgamt[i])
    current_pricipal2[i] = np.fv(retailloan_rate[i], retailloan_times[i], m2[i], -retailloan_orgamt[i])
    retailloan_EAD[i] = current_pricipal2[i]
    retailloan_provision[i] = retailloan_PD[i] * retailloan_LGD * current_pricipal2[i]


In [11]:
print(np.sum(retailloan_provision))

110867480.10541177


In [12]:
#Part 1b
#Capital Requirements of bond
from scipy.stats import norm

R_bond = np.zeros(n)
K_bond = np.zeros(n)
b = np.zeros(n)
M = np.zeros(n)
maturity_month = np.zeros(n)
val1 = 0
val2 = 0
last_term = np.zeros(n)
for i in range (n):
    R_bond[i] =  0.12 * ( (1 - np.exp(-50 * bond_PD[i])) / (1 - np.exp(-50)) ) 
    R_bond[i] += 0.24 * (1 - ( (1 - np.exp(-50 * bond_PD[i])) / (1 - np.exp(-50)) ) )
    K_bond[i] = norm.cdf(np.sqrt( (1 - R_bond[i]) ** (-1) ) * norm.ppf(bond_PD[i]) + 
               np.sqrt( R_bond[i] / (1 - R_bond[i])) * norm.ppf(0.999) ) - bond_PD[i]
    K_bond[i] *= bond_LGD[i]
    b[i] = (0.11852 - 0.05478*np.log(bond_PD[i]))**2
    number_coupon_year[i] = 365/((365*time_maturity[i])/coupon_remain[i])
    coupon_rate[i] = yearly_rate[i]/ number_coupon_year[i]
    coupon[i] = coupon_rate[i] * principal[i]
    payment_series = np.repeat(coupon[i], coupon_remain[i]+1)
    payment_series[0] -= coupon[i]
    payment_series[coupon_remain[i]] += principal[i]
    #maturity_month[i] = np.floor(time_maturity[i] *12)
    for j in range(coupon_remain[i]):
        val1 += j*payment_series[j]
        val2 += payment_series[j]
        M[i] = val1/val2
    last_term[i] = (1+(M[i]-2.5)*b[i])/(1-1.5*b[i])
    K_bond[i] *= last_term[i]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
print(K_bond)

[0.43849018 0.46023952 0.38681421 ... 0.39503463 0.4269955  0.36443615]


In [14]:
#Part 1b Capital Requirements of mortgage
K_mortgage = np.zeros(mortgage_len)
for i in range (mortgage_len):
    R_mortgage =  0.15
    K_mortgage[i] = norm.cdf(np.sqrt( (1 - R_mortgage) ** (-1) ) * norm.ppf(mortgage_PD[i]) + 
                          np.sqrt( R_mortgage / (1 - R_mortgage) ) * norm.ppf(0.999) ) - mortgage_PD[i]
    K_mortgage[i]*= mortgage_LGD[i]

In [15]:
print(np.sum(K_mortgage))

3658.800797118064


In [16]:
#Part 1b Capital Requirements of retailoan
K_retailloan = np.zeros(retailloan_len)
R_retailloan = np.zeros(retailloan_len)
for i in range (retailloan_len):
    R_retailloan[i] =  0.03 * ( (1 - np.exp(-35 * retailloan_PD[i])) / (1 - np.exp(-35)) ) 
    R_retailloan[i] += 0.16 * (1 - ( (1 - np.exp(-35 * retailloan_PD[i])) / (1 - np.exp(-35)) ) )
    K_retailloan[i] = norm.cdf(np.sqrt( (1 - R_retailloan[i]) ** (-1) ) * norm.ppf(retailloan_PD[i]) + 
                          np.sqrt( R_mortgage / (1 - R_retailloan[i]) ) * norm.ppf(0.999) ) - retailloan_PD[i]
    K_retailloan[i] *= retailloan_LGD

In [17]:
print(np.sum(K_retailloan))

5481.168601357005


In [18]:
#Part1 c Risk weighted assets
#Bond
RW_bond = np.zeros(n)
RWA_bond = 0
for i in range(n):
    if bond_PD[i] < 0.01:
        RW_bond[i] = 0
    if 0.01 < bond_PD[i] < 0.12:
        RW_bond[i] = 0.2
    if 0.12 < bond_PD[i] < 0.36:
        RW_bond[i] = 0.5
    if 0.36 < bond_PD[i] < 0.75:
        RW_bond[i] = 1
    if bond_PD[i] > 0.75:
        RW_bond[i] = 1.5
    RWA_bond += RW_bond[i] * bond_EAD[i]
print(RWA_bond)

14444330147.809198


In [19]:
#Part1 c Risk weighted assets
#Mortgage
RWA_mortgage = 0
for i in range(mortgage_len):
    RW_mortgage = 0.35
    RWA_mortgage += RW_mortgage * mortgage_EAD[i]
print(RWA_mortgage)

17863711919.030434


In [20]:
#Part1 c Risk weighted assets
#Mortgage
RWA_loan = 0
for i in range(retailloan_len):
    RW_loan = 0.75
    RWA_loan += RW_loan *  retailloan_EAD[i]
print(RWA_loan)

332456509.37822986


In [21]:
#Regulatory Capital 
regulatory_capital = 0.115* (RWA_bond+RWA_mortgage+RWA_loan)

In [22]:
print(regulatory_capital) 

3753657336.265054
